In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## <span style = 'background :lightblue'>STEP 1 : Import Transforrmed Dataset</span>

In [2]:
filepath_train = r"S3_FinalModel_Part2_FE_Pipeline_Train_DS.csv"
filepath_test =  r"S3_FinalModel_Part2_FE_Pipeline_Test_DS.csv"

In [3]:
d_train = pd.read_csv(filepath_train  ,index_col= 0 , header= 0)
d_test = pd.read_csv(filepath_test  ,index_col= 0 , header= 0)

In [4]:
d_train.head(3)

,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_NO,SeniorCitizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,-0.338701,0.049061,0,1,1,0,0,1,1,0,...,0,0,1,1,0,1,0,0,0,0
1,1.030607,0.139714,1,0,1,0,1,0,1,0,...,1,0,0,0,1,1,0,0,0,0
2,0.147022,-0.956056,1,0,1,0,1,0,1,0,...,1,0,0,1,0,0,0,1,0,1


In [5]:
d_test.head(3)

,MonthlyCharges,TotalCharges,gender_Female,gender_Male,SeniorCitizen_NO,SeniorCitizen_Yes,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,-1.457688,-0.536472,0,1,1,0,0,1,1,0,...,0,0,1,1,0,0,0,0,1,0
1,-1.480896,-0.374264,0,1,1,0,1,0,1,0,...,0,0,1,1,0,1,0,0,0,0
2,0.142049,0.810810,0,1,1,0,1,0,1,0,...,0,1,0,1,0,1,0,0,0,0


In [6]:
import warnings
warnings.filterwarnings('ignore')

## <span style = 'background :lightblue'>STEP 2 : Train - Test Split</span>

In [7]:
X_train = d_train.drop(labels= ['Churn'] , axis = 1)
y_train = d_train['Churn']

X_test = d_test.drop(labels= ['Churn'] , axis = 1)
y_test = d_test['Churn']

In [8]:
X_train.shape , y_train.shape

((5634, 51), (5634,))

In [9]:
X_test.shape , y_test.shape

((1409, 51), (1409,))

In [10]:
y_train.value_counts()

0    4149
1    1485
Name: Churn, dtype: int64

In [11]:
y_test.value_counts()

0    1025
1     384
Name: Churn, dtype: int64

## <span style = 'background :lightblue'>STEP 3 : Up-sampling of Train and Test DataSet</span>

In [12]:
from imblearn.combine import SMOTEENN

In [13]:
def SmoteENN(X_DF , y_ser ,samp_strategy):
    
    # Define KMeansSMOTE object with sampling_strategy
    senn = SMOTEENN(random_state = 100 ,sampling_strategy=samp_strategy  )
    
    # fit and apply the transform
    X_DF_senn_res, y_ser_senn_res = senn.fit_resample(X_DF, y_ser)
    
    return (X_DF_senn_res, y_ser_senn_res)

In [14]:
X_train_res, y_train_res = SmoteENN(X_DF = X_train , y_ser = y_train , samp_strategy = 0.95 )

X_test_res , y_test_res = SmoteENN(X_DF = X_test ,  y_ser = y_test , samp_strategy = 0.85)

In [15]:
X_train_res.shape

(4978, 51)

In [16]:
y_train_res.shape

(4978,)

In [17]:
y_train_res.value_counts()

1    2828
0    2150
Name: Churn, dtype: int64

In [18]:
X_test_res.shape

(1127, 51)

In [19]:
y_test_res.shape

(1127,)

In [20]:
y_test_res.value_counts()

1    617
0    510
Name: Churn, dtype: int64

## <span style = 'background :lightblue'>STEP 4 : Training models</span>

In [21]:
from sklearn.metrics import classification_report

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [23]:
models = [LogisticRegression(max_iter=1000), SVC(), GaussianNB(),DecisionTreeClassifier(), RandomForestClassifier()]

In [24]:
def ModelTraining(ML_models , X_train ,y_train ,X_test,y_test):
    
    for model in ML_models :

        model.fit(X_train_res.values, y_train_res)
        y_hat_train = model.predict(X_train_res.values)
        y_hat_test = model.predict(X_test_res.values)

        print(f"******************* \33[1m{str(model)}\33[0m *******************\n")

        print("\33[1mTraining classification Report :\33[0m \n")
        print(classification_report(y_train_res,y_hat_train ,zero_division = 1))

        print() 

        print("\33[1mTesting classification Report :\33[0m \n")
        print(classification_report(y_test_res,y_hat_test , zero_division = 1))

        print()
        

In [25]:
# X_train_res, y_train_res ,X_test_res , y_test_res

In [26]:
ModelTraining(ML_models = models ,  X_train = X_train_res ,y_train = y_train_res ,
              X_test = X_test_res, y_test = y_test_res)

******************* LogisticRegression(max_iter=1000) *******************

Training classification Report : 

              precision    recall  f1-score   support

           0       0.94      0.95      0.94      2150
           1       0.96      0.96      0.96      2828

    accuracy                           0.95      4978
   macro avg       0.95      0.95      0.95      4978
weighted avg       0.95      0.95      0.95      4978


Testing classification Report : 

              precision    recall  f1-score   support

           0       0.91      0.95      0.93       510
           1       0.95      0.92      0.94       617

    accuracy                           0.93      1127
   macro avg       0.93      0.93      0.93      1127
weighted avg       0.93      0.93      0.93      1127


******************* SVC() *******************

Training classification Report : 

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      2150
         

## Hyperparameter Tuning 

### Best Performed Model : Logistic Regression

In [27]:
from sklearn.model_selection import GridSearchCV

In [28]:
log_reg = LogisticRegression()

In [30]:
Param_grid = [
                {
                    "solver": ['newton-cg', 'lbfgs'],
                    "C": [100, 10, 1.0, 0.1, 0.01]
                }
             ]

In [33]:
def GridCV(Model , Param_Grid ,Scoring , CV ,X_train ,y_train ,X_test ,y_test):
    
    # Creating object of GridSearchCV class
    grid_search = GridSearchCV(estimator = Model, param_grid =  Param_Grid, 
                               scoring = Scoring , cv = CV)
    
    # Fitting to the object
    grid_search.fit(X_train.values , y_train)
    
    # Printing the Best parameters
    print("Grid Search CV Best Parameters : ",grid_search.best_params_)
    
    # Mean cross-validated score of the best_estimator
    print("Grid Search CV Best Score : ",grid_search.best_score_,"\n")
    
    # Best Fitted Model
    best_model = grid_search.best_estimator_
    
    # Prediction : Trian and Test
    y_hat_GSCV_train = best_model.predict(X_train.values)
    y_hat_GSCV_test = best_model.predict(X_test.values)
    
    print("\33[1mTraining classification Report :\33[0m \n")
    print(classification_report(y_train,y_hat_GSCV_train ,zero_division = 1))

    print() 

    print("\33[1mTesting classification Report :\33[0m \n")
    print(classification_report(y_test,y_hat_GSCV_test , zero_division = 1))

    print()
    

In [34]:
GridCV(Model = log_reg , Param_Grid = Param_grid ,Scoring = "accuracy" , CV = 10 ,
       X_train = X_train_res ,y_train = y_train_res , X_test = X_test_res, y_test = y_test_res)
             

Grid Search CV Best Parameters :  {'C': 100, 'solver': 'newton-cg'}
Grid Search CV Best Score :  0.9509918951459764 

Training classification Report : 

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2150
           1       0.96      0.96      0.96      2828

    accuracy                           0.95      4978
   macro avg       0.95      0.95      0.95      4978
weighted avg       0.95      0.95      0.95      4978


Testing classification Report : 

              precision    recall  f1-score   support

           0       0.90      0.95      0.93       510
           1       0.95      0.92      0.94       617

    accuracy                           0.93      1127
   macro avg       0.93      0.93      0.93      1127
weighted avg       0.93      0.93      0.93      1127


